In [ ]:
#Load Packages
library(DESeq2)
library(edgeR)
library(limma)
library(gplots)
library(RColorBrewer)
library(pheatmap)
library(ggplot2)
library(ggrepel)
library(pathfindR)
library(scales)
library(data.table)
library(fBasics)
library(forcats)
library(omu)
library(maptools)
library(phyloseq)
library(vegan)

In [ ]:
#Set Theme for Figures
theme<-theme(panel.background = element_blank(),panel.border=element_rect(fill=NA),
        panel.grid.major = element_blank(),panel.grid.minor = element_blank(),strip.background=element_blank(),
        axis.text.x=element_text(colour="black"),axis.text.y=element_text(colour="black"),
        axis.ticks=element_line(colour="black"),plot.margin=unit(c(1,1,1,1),"line"), legend.position="none")

In [ ]:
#Choose Alpha/FDR
alpha = 0.01

In [ ]:
#Load Meta Data
coldata <- read.delim2("Map.Metatrancriptome.COPD.SmNV.b2.txt", sep="\t")

#Order Meta Data by SampleId
coldata <- coldata[order(coldata$SampleID),]

In [ ]:
#load Count Data
mycounts <-read.delim2("Metagenome_KEGG_ANNOTATED.txt", sep="\t", row.names=1)

#Order Count Data by SampleID
mycounts <-mycounts[, order(colnames(mycounts))]

#Confirm Sample IDs match for Count and Meta Data
table(colnames(mycounts)==as.character(coldata$SampleID))

In [ ]:
#Fix Zeros and Change date to Numeric/Integer


#Convert any NAs to 0
mycounts[is.na(mycounts)] <- 0

#Copy of Count Table
mycounts3 <- mycounts

#Convert Count Table into a Numeic Data Frame
d1 = data.frame(lapply(mycounts3, function(x) as.numeric(as.character(x))),
                   check.names=F, row.names = rownames(mycounts3))

#Convert Data to Integers to Run DESEq
d1[] <- lapply(d1, as.integer)

In [ ]:
#Make CountData and MetaData into DESEq Object; Choose the comparison Variable as design
dds <- DESeqDataSetFromMatrix(countData = d1,
                              colData = coldata,
                              design= ~ Sample_Type_DMM_Class_ReSeq)

In [ ]:
#Calculate geometric means prior to estimate size factor
gm_mean = function(x, na.rm=TRUE){ exp(sum(log(x[x > 0]), na.rm=na.rm) / length(x))}
geoMeans = apply(counts(dds), 1, gm_mean)

#Estimate Factors of DESeq Object
dds <- estimateSizeFactors(dds, geoMeans = geoMeans)

In [ ]:
#----------------------
##Create graph of Reads
#----------------------
#Sum number or reads per sample
summary <- as.data.frame(rowSums(t(assay(dds))))
#Merge Reads Data with MetaData
require(data.table)
Reads <- as.data.frame(merge(x = summary, y = colData(dds), by = "row.names", all.x = TRUE))
#Rename Column of Reads
colnames(Reads)[colnames(Reads)=="rowSums.t.assay.dds..."] <- "Reads"
#Set Order Of Figure
Reads$or <-ifelse(Reads$Sample_Type_DMM_Class_ReSeq=="BKG", 1,NA)
Reads$or <-ifelse(Reads$Sample_Type_DMM_Class_ReSeq=="BAL.BPT",2 ,Reads$or)
Reads$or <-ifelse(Reads$Sample_Type_DMM_Class_ReSeq=="BAL.SPT",3 ,Reads$or)
Reads$or <-ifelse(Reads$Sample_Type_DMM_Class_ReSeq=="Sup",4 ,Reads$or)
#Create Figure
    ggplot(Reads, aes(x= reorder(Sample_Type_DMM_Class_ReSeq, +or), y=Reads, fill=Sample_Type_DMM_Class_ReSeq)) + 
    stat_boxplot(geom ='errorbar', width=0.1)+
    geom_boxplot(outlier.shape = NA, width=0.5)+
    geom_jitter(shape=1, position=position_jitter(0.2))+
    scale_fill_manual(values=c("#296218", "#EA3323", "#000000","#932CE7")) +
    scale_x_discrete(labels = c('BKG','BPT','SPT','Sup'))+ 
    scale_y_continuous(name="Reads",trans="log10", breaks = trans_breaks('log10', function(x) 10^x), labels = trans_format('log10', math_format(10^.x))) +
    xlab("Sample Type")+
    theme
                                                                         
#Run Statistics                                                                        
kruskal.test(Reads ~ Sample_Type_DMM_Class_ReSeq, data = Reads)

In [ ]:
#Transforming data - Option #1 is regularized-logarithm transformation, or rlog for short. 
#rld <- rlog(dds, fitType="local")

#Transforming data - Option #2 Variance Stabilizing
vsd <- varianceStabilizingTransformation(dds)

In [ ]:
#Removed ungrouped KEGGS
dds1 <- dds[!grepl("UNGROUPED|UNMAPPED",rownames(assay(dds))),]
rld1 <- rld[!grepl("UNGROUPED|UNMAPPED",rownames(assay(rld))),]
vsd1 <- vsd[!grepl("UNGROUPED|UNMAPPED",rownames(assay(vsd))),]

#keep only data which includes Taxa Data
dds2 <- dds1[grep("g_|unclassified",rownames(assay(dds1))),]
dds2 <- dds2[!grepl("unclassified",rownames(assay(dds2))),]
rld2 <- rld1[grep("g_|unclassified",rownames(assay(rld1))),]
rld2 <- rld2[!grepl("unclassified",rownames(assay(rld2))),]
vsd2 <- vsd1[grep("g_|unclassified",rownames(assay(vsd1))),]
vsd2 <- vsd2[!grepl("unclassified",rownames(assay(vsd2))),]

#keep only data for KEGG without Taxa Data
dds3 <- dds1[!grepl("g_|unclassified",rownames(assay(dds1))),]
rld3 <- rld1[!grepl("g_|unclassified",rownames(assay(rld1))),]
vsd3 <- vsd1[!grepl("g_|unclassified",rownames(assay(vsd1))),]

#===========================================================================================================================================================================
#===========================================================================================================================================================================
#////////////////// KEGG /////////////////////
#===========================================================================================================================================================================
#===========================================================================================================================================================================

In [ ]:
#=========================================================
#/////////////PCOA PLOT based on 16S Clustering///////////
#=========================================================
#Fix the Negative Values
#rld30 <- ifelse(assay(rld3)<0,0,assay(rld3))
vsd30 <- ifelse(assay(vsd3)<0,0,assay(vsd3))

#Create Distance Matrix
#vegdist   = vegdist(t(rld30), method="bray")
vegdist   = vegdist(t(vsd30), method="bray")

#Formulate principal component co-ordinates for PCOA plot, k as the choice of PCs
CmdScale <- cmdscale(vegdist, k =10)
#calculated Sample variance for each PC
vars <- apply(CmdScale, 2, var)
#Create Variable with the Percent Variance
percentVar <- round(100 * (vars/sum(vars)))

#Merge PC Data with MetaData
require(data.table)
newResults <- merge(x = CmdScale, y = colData(vsd3), by = "row.names", all.x = TRUE)
#Rename Variables for PC1 and PC2
colnames(newResults)[colnames(newResults)=="V1"] <- "PC1"
colnames(newResults)[colnames(newResults)=="V2"] <- "PC2"
colnames(newResults)[colnames(newResults)=="Row.names"] <- "name"

#Calculate the Centroid Value
centroids <- aggregate(cbind(PC1,PC2)~Sample_Type_DMM_Class_ReSeq,data= newResults, mean)
#Merge the Centroid Data into the PCOA Data
newResults <- merge(newResults,centroids,by="Sample_Type_DMM_Class_ReSeq",suffixes=c("",".centroid"))

#Plot PCOA 
    ggplot(newResults, aes(PC1, PC2, color=Sample_Type_DMM_Class_ReSeq)) +
    geom_point(size=5, alpha=0.7) +
    xlab(paste0("PC1: ",percentVar[1],"% variance")) +
    ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
    scale_color_manual(values=c("#296218", "#EA3323", "#000000","#932CE7")) + 
    geom_point(data=centroids, aes(x=PC1, y=PC2, color=Sample_Type_DMM_Class_ReSeq), size=0) +
    geom_segment(aes(x=PC1.centroid, y=PC2.centroid, xend=PC1, yend=PC2, color=Sample_Type_DMM_Class_ReSeq))+ 
    geom_label_repel(data = centroids, aes(x=PC1, y=PC2, label=c("BAL.BPT", "BAL.SPT", "BKG", "UA")), size=10) +
    theme(panel.background = element_blank(),panel.border=element_rect(fill=NA),panel.grid.major = element_blank(),panel.grid.minor = element_blank(),strip.background=element_blank(),axis.title=element_text(size=30,face="bold"),axis.text.x=element_blank(),axis.text.y=element_blank(),axis.ticks=element_blank(),plot.margin=unit(c(1,1,1,1),"line"), legend.position="none")

#Run Statistics
adonis(vegdist ~ vsd3$Sample_Type_DMM_Class_ReSeq)